In [1]:
%load_ext autoreload
%autoreload 2

%cd ..

/Users/rubenbroekx/Documents/Projects/radix-co2-reduction


# Analyse Field

Analyse a single field using its coordinates and Google Earth Engine satellite imagery.

In [8]:
import re
import ee
import json
import folium
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from datetime import datetime
from pathlib import Path

from src.radix_co2_reduction.earth_engine.visualisation import create_map, show_polygon, show_point
from src.radix_co2_reduction.earth_engine.datasets import CroplandCollection, Landsat7Collection, Landsat8Collection, Sentinel2Collection, NaipCollection
from src.radix_co2_reduction.earth_engine.session import start
from src.radix_co2_reduction.earth_engine.utils import create_bounding_box, create_polygon, download_as_png, to_polygon

In [3]:
# Start a Earth Engine session
start()

Enter verification code: 4/1AY0e-g5MTLNm3p0XFa2WWN7aaFQn0shvVujo1Vr1eVWgtKx1KrShqb8_Bic

Successfully saved authorization token.


In [4]:
# Load in Beck's data
beck = pd.read_csv(Path.cwd() / 'data/beck_corrected.csv')

# Load in all field-boundaries
with open(Path.cwd() / 'data/polygons.json', 'r') as f:
    boundaries = json.load(f)

In [5]:
def extract_harvest_date(d: str) -> str:
    """Parse out the harvest date."""
    d = re.sub(r'[\n]+', ' ', d)[11:21]
    try:
        return datetime.strptime(d, '%m/%d/%Y').strftime('%Y-%m-%d')
    except ValueError:
        return None


def extract_planting_date(d: str) -> str:
    """Parse out the harvest date."""
    d = re.sub(r'[\n]+', ' ', d)[9:19]
    try:
        return datetime.strptime(d, '%m/%d/%Y').strftime('%Y-%m-%d')
    except ValueError:
        return None

In [36]:
def analyse(field_id):
    # Load in field information
    field = beck[beck['id'] == field_id].iloc[0]

    ID = int(field.id)
    YEAR = int(field.year)
    print(f" - Field from year: {YEAR}")
    TILL_TYPE = field.tillage[9:]
    print(f" - Tillage type: {TILL_TYPE}")
    harvest_date = extract_harvest_date(field.harvest_date)
    print(f" - Harvesting date: {harvest_date}")
    planting_date = extract_planting_date(field.planted_date)
    print(f" - Planting date: {planting_date}")
    startdate = f"{YEAR - 1}-11-01"
    enddate = f"{YEAR}-06-30"
    print(f" - Extracting data from {startdate} until {enddate}")
    
    # Create bounding box around field and load in field's polygon
    bounding_box = to_polygon(create_bounding_box(
            lng=field.lng,
            lat=field.lat,
            offset=1280,
    ))
    coordinates = boundaries[str(ID)]
    field_boundary = create_polygon([coordinates])

    # Load in all datasets
    landsat7 = Landsat7Collection()
    landsat7.load_collection(
            region=field_boundary,
            startdate=startdate,
            enddate=enddate,
            filter_clouds=True,
            return_masked=False,
            filter_perc=.75,
            relevant_bands=['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'],
    )
    print(f" - Number of data samples for {landsat7}: {landsat7.get_size()}")

    # Landsat 8 data: https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR
    landsat8 = Landsat8Collection()
    landsat8.load_collection(
            region=field_boundary,
            startdate=startdate,
            enddate=enddate,
            filter_clouds=True,
            return_masked=False,
            filter_perc=.25,
            relevant_bands=['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa'],
    )
    print(f" - Number of data samples for {landsat8}: {landsat8.get_size()}")

    # # Sentinel-2 L2A data: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR
    sentinel2 = Sentinel2Collection()
    sentinel2.load_collection(
            region=field_boundary,
            startdate=startdate,
            enddate=enddate,
            filter_clouds=True,
            return_masked=False,
            filter_perc=.25,
            relevant_bands=['B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'QA60', 'SCL'],
    )
    print(f" - Number of data samples for {sentinel2}: {sentinel2.get_size()}")

    # USDA Cropland data: https://developers.google.com/earth-engine/datasets/catalog/USDA_NASS_CDL
    cropland = CroplandCollection()
    cropland.load_collection(
            region=field_boundary,
            startdate=f'{YEAR}-01-01',
            enddate=f'{YEAR}-12-31',
    )
    
    # NAIP data: https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ
    naip = NaipCollection()
    naip.load_collection(
            region=field_boundary,
            startdate=f'2017-01-01',
            enddate=f'2020-12-31',
    )
    
    # Show on map
    mp = create_map(coordinate=(field.lat, field.lng), zoom=14)
#     mp = landsat7.add_to_map(mp, scheme='all')
#     mp = landsat8.add_to_map(mp, scheme='all')
#     mp = sentinel2.add_to_map(mp, scheme='all')
#     mp = cropland.add_to_map(mp, scheme='first')
    mp = naip.add_to_map(mp, scheme='last')
    mp = show_point(mp, point=ee.Geometry.Point(field.lng, field.lat))
    mp = show_polygon(mp, polygon=bounding_box, tag='Bounding Box')
    mp = show_polygon(mp, polygon=field_boundary, tag='Field')
    mp.add_child(folium.LayerControl())
    display(mp)

In [11]:
analyse(103956)

 - Field from year: 2020
 - Tillage type: l
 - Harvesting date: None
 - Planting date: None
 - Extracting data from 2019-11-01 until 2020-06-30
 - Number of data samples for Landsat 7 SR-T1: 4
 - Number of data samples for Landsat 8 SR-T1: 8
 - Number of data samples for Sentinel-2 L2A SR: 21


In [37]:
analyse(104400)

 - Field from year: 2020
 - Tillage type: Min.-Till
 - Harvesting date: 2020-09-17
 - Planting date: 2020-04-21
 - Extracting data from 2019-11-01 until 2020-06-30
 - Number of data samples for Landsat 7 SR-T1: 7
 - Number of data samples for Landsat 8 SR-T1: 7
 - Number of data samples for Sentinel-2 L2A SR: 21


In [23]:
beck = pd.read_csv(Path.cwd() / 'data/beck_corn_data.csv', index_col=0)
d = {}
for i in beck.id:
    sdf = beck[beck.id == i].iloc[0]
    d[int(i)] = f"{sdf.lat}-{sdf.lng}"

In [39]:
ids = [104394,
 104402,
 104400,
 104373,
 104404,
 104550,
 104406,
 104403,
 104405,
 104551,
 102759,
 102764,
 102797,
 102802,
 102816,
 102819,
 102823,
 102828,
 102832,
 102846,
 102849,
 102860,
 102871,
 102878,
 102921,
 102959,
 102968,
 102977,
 102979,
 102990,
 103910,
 103956,
 103912,
 103916,
 103914,
 103917,
 103911,
 103902,
 103915,
 103900]

In [40]:
d_relevant = {}
for i in ids:
    d_relevant[i] = d[i]
d_relevant

{104394: '39.6679328199836--95.4287818841267',
 104402: '41.8049052687692--89.9462267030189',
 104400: '37.2292207864713--86.8936640526172',
 104373: '40.5537664309943--86.5148327618408',
 104404: '41.2411259521817--82.9936408996582',
 104550: '38.8381287601523--95.04570706051',
 104406: '40.7992566200523--81.7018890380859',
 104403: '41.3497073605845--89.8826967564129',
 104405: '41.2705736167006--82.6459836959839',
 104551: '39.2359428773568--91.8431282043457',
 102759: '40.7503130678784--87.0291101932526',
 102764: '40.6059378740043--87.0167398452759',
 102797: '38.2517293230358--87.4965763092041',
 102802: '44.0346771143346--93.8702092820737',
 102816: '43.7839925305993--92.4227786633144',
 102819: '35.3087866338525--87.4896240234375',
 102823: '44.6139945188537--95.4755313826988',
 102828: '39.2794239182597--87.6673579216003',
 102832: '43.9847601324212--96.5876326807513',
 102846: '39.5487283498371--88.3416438102722',
 102849: '43.0237255888202--95.8220672607422',
 102860: '44.68

In [141]:
s_landsat7 = landsat7.sample(
    region=field_boundary,
)
s_landsat8 = landsat8.sample(
    region=field_boundary,
)
s_sentinel2 = sentinel2.sample(
    region=field_boundary,
)

Sampling Sentinel-2 L2A SR: 100%|██████████| 14/14 [00:17<00:00,  1.26s/it]


## Collect Cloud Data

The following script is used to collect data for the cloud-filter.

In [34]:
# cloud_data = {'cloudy':{}, 'clear':{}}  # TODO: comment for precaution

In [145]:
# Add as clouds
dataset = 'sentinel2'
date = '2020-05-26'

if dataset == 'landsat7':
    samples = s_landsat7
elif dataset == 'landsat8':
    samples = s_landsat8
elif dataset == 'sentinel2':
    samples = s_sentinel2
else:
    raise Exception
assert date in samples
cloud_data['cloudy'][f"{FIELD_ID}-{date}-{dataset}"] = samples[date]

len(cloud_data['cloudy']),len(cloud_data['clear'])

(38, 395)

In [146]:
# Add all the others as none-clouds
for dataset in ('landsat7', 'landsat8', 'sentinel2'):
    if dataset == 'landsat7':
        samples = s_landsat7
    elif dataset == 'landsat8':
        samples = s_landsat8
    elif dataset == 'sentinel2':
        samples = s_sentinel2
    for date, sample in samples.items():
        if f"{FIELD_ID}-{date}-{dataset}" in cloud_data['cloudy']: continue
        cloud_data['clear'][f"{FIELD_ID}-{date}-{dataset}"] = sample
    
len(cloud_data['cloudy']),len(cloud_data['clear'])

(38, 414)

In [147]:
# with open(Path.cwd() / 'data/cloud_data.json', 'w') as f:
#     json.dump(cloud_data, f)